In [1]:
library(tidyverse)
library(magrittr)
library(here)
library(bdkn)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.0       ✔ purrr   0.3.0  
✔ tibble  2.0.1       ✔ dplyr   0.8.0.1
✔ tidyr   0.8.2       ✔ stringr 1.4.0  
✔ readr   1.3.1       ✔ forcats 0.4.0  
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘magrittr’

The following object is masked from ‘package:purrr’:

    set_names

The following object is masked from ‘package:tidyr’:

    extract

here() starts at /Users/martin_petr/projects/ychr

Attaching package: ‘bdkn’

The following object is masked from ‘package:ggplot2’:

    %+%



In [2]:
devtools::load_all(".")

Loading ychr


In [12]:
gt <- read_simgt("../out.tsv")

In [13]:
samples <- read_siminfo(gt)

In [14]:
tAfrRef <- calculate_tAfrRef(gt, samples) %>% select(afr, emh, starts_with("tmrca"), mut_rate, everything())

In [15]:
tAfrRef %>% group_by(afr) %>% summarise_if(is.numeric, mean)

afr,tmrca_ad,tmrca_f,tmrca_avg,mut_rate,a,b,c,d,e,f,total,age,muts_per_year
afr0,217290.2,225891.6,221590.9,3.754053e-11,44,0,0,12.8,1,59,6985043,45000,0.0002622222
afr1,217290.2,225891.6,221590.9,3.754053e-11,44,0,0,12.8,1,59,6985043,45000,0.0002622222
afr2,217290.2,229720.3,223505.2,3.754053e-11,44,0,0,12.8,1,60,6985043,45000,0.0002622222
afr3,217290.2,225891.6,221590.9,3.754053e-11,44,0,0,12.8,1,59,6985043,45000,0.0002622222
afr4,217290.2,225891.6,221590.9,3.754053e-11,44,0,0,12.8,1,59,6985043,45000,0.0002622222


In [24]:
tAfrRef %>% group_by(afr) %>% summarise_if(is.numeric, mean)

afr,tmrca_ad,tmrca_f,tmrca_avg,mut_rate,a,b,c,d,e,f,total,age,muts_per_year
afr0,239010.1,235958.2,237484.1,3.269892e-10,439,0,0,108,5,540,6999891,45000,0.002288889
afr1,239010.1,235084.3,237047.2,3.269892e-10,439,0,0,108,5,538,6999891,45000,0.002288889
afr2,239010.1,235958.2,237484.1,3.269892e-10,439,0,0,108,5,540,6999891,45000,0.002288889
afr3,239010.1,233336.4,236173.2,3.269892e-10,439,0,0,108,5,534,6999891,45000,0.002288889
afr4,239010.1,235958.2,237484.1,3.269892e-10,439,0,0,108,5,540,6999891,45000,0.002288889


In [25]:
tArchRef <- calculate_tArchRef(gt, samples, tAfrRef)

In [26]:
tArchRef %>% group_by(arch) %>% summarise_if(is.numeric, mean)

arch,tmrca_arch,alpha,mut_rate,tmrca_ad,tmrca_f,tmrca_avg,a,b,c,d,e,f,total,p
arch0,636078.7,2.682339,3.269892e-10,239010.1,235259,237134.6,913,0,0,547,538.4,1159,6999891,0.4568663
arch1,636078.7,2.682339,3.269892e-10,239010.1,235259,237134.6,913,0,0,547,538.4,1348,6999891,0.4568663
